In [15]:
import random
import cvxpy as cp
import numpy as np

In [16]:
i = 3  # number of cars
j = 5  # number of incidents

b = [random.randint(1, 3) for i in range(j)]

x = cp.Variable((i, j), boolean=True)
z = cp.Variable(j, boolean=True)

objective = cp.Maximize(cp.sum(b * z))
constraints = [cp.sum(x, axis=0) >= b * z, cp.sum(x, axis=1) <= 1]
prob = cp.Problem(objective, constraints)


C:\Users\itrmi\AppData\Local\Programs\Python\Python310\lib\site-packages\cvxpy\expressions\expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 7 times so far.

  warnings.warn(msg, UserWarning)
C:\Users\itrmi\AppData\Local\Programs\Python\Python310\lib\site-packages\cvxpy\expressions\expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 8 times so 

In [17]:
prob.solve(solver="GLPK_MI")

0.0

In [18]:
x.value

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.]])

In [19]:
z.value

array([0., 0., 0., 0., 0.])

In [20]:
b

[2, 1, 1, 3, 1]